#Import Data
I will drop this comment later.

Here, Professor, I reduced the size of the two datasets because the performance of classical supervised methods (Without TL) is so high and it is so difficult to see an improvement with TL.

By reducing the size of the datasets I obtained a target model with acceptable performance and by using TL the performance of the model *increased* significantly. 

In [1]:
import pandas as pd
Final_Data_S1 = pd.read_csv("/content/drive/MyDrive/Final_Data_S1.csv")
Final_Data_S2 = pd.read_csv("/content/drive/MyDrive/Final_Data_S2.csv")

In [2]:
Final_Data_S1.loc[Final_Data_S1[Final_Data_S1.labels.isin([3.0])].index,'labels']=2.0
Final_Data_S2.loc[Final_Data_S2[Final_Data_S2.labels.isin([3.0])].index,'labels']=2.0

In [3]:
Final_Data_S1.loc[Final_Data_S1[Final_Data_S1.labels.isin([4.0])].index,'labels']=2.0
Final_Data_S2.loc[Final_Data_S2[Final_Data_S2.labels.isin([4.0])].index,'labels']=2.0

In [4]:
Data_0 = Final_Data_S1[Final_Data_S1.labels == 0].iloc[:386]
Data_1 = Final_Data_S1[Final_Data_S1.labels == 1].iloc[:95]
Data_2 = Final_Data_S1[Final_Data_S1.labels == 2].iloc[:52]
frames = [Data_0, Data_1, Data_2]
Final_Data_S1 = pd.concat(frames)

In [5]:
Data_0 = Final_Data_S2[Final_Data_S2.labels == 0].iloc[:772]
Data_1 = Final_Data_S2[Final_Data_S2.labels == 1].iloc[:190]
Data_2 = Final_Data_S2[Final_Data_S2.labels == 2].iloc[:104]
frames = [Data_0, Data_1, Data_2]
Final_Data_S2 = pd.concat(frames)

In [6]:
Final_Data_S1.loc[Final_Data_S1[Final_Data_S1.labels.isin([2.0])].index,'labels']=1.0
Final_Data_S2.loc[Final_Data_S2[Final_Data_S2.labels.isin([2.0])].index,'labels']=1.0

#Class Transformation
We created a PCA-like transformation for the source and for the target datasets to transform the data into a new common feature space.

In [7]:
import numpy as np 
import pandas as pd 

np.random.seed(8)


class Transformation:


  def remove_rows(self, Final_Data):
    Final_Data = Final_Data[Final_Data.labels != 4.0]
    return Final_Data

  def remove_features(self, Final_Data):
    Final_Data = Final_Data.drop(['time'], axis = 1)
    return Final_Data

  def centered_data(self, data):
    for i in range(0,data.shape[1] - 1):
      mean = data.iloc[:,i].mean()
      data.iloc[:,i] = data.iloc[:,i] - mean
    return data

  def Transoform_Data(self, data, X, n_pca, name, number_classes):

    data_prob = data.labels.value_counts()/len(data)
    indx_prob = {i:data_prob[i] for i in data_prob.index}

    initial_class = pd.DataFrame(data = data.groupby('labels').cov().iloc[0:data.shape[1]-1]).index[0][0]
    cov = (data.groupby('labels').cov().iloc[0:data.shape[1]-1])*indx_prob[initial_class]

    for i in range(data.shape[1]-1,number_classes*(data.shape[1]-1),data.shape[1]-1):
      next_class = pd.DataFrame(data = data.groupby('labels').cov().iloc[i:i+data.shape[1]-1]).index[0][0]
      cov += (data.groupby('labels').cov().iloc[i:i+data.shape[1]-1].values)*indx_prob[next_class]

    cov = pd.DataFrame(data = cov)
    eigenValues, eigenVectors = np.linalg.eig(cov)
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]

    
    Cov_Matrix = pd.DataFrame(data = eigenVectors).iloc[:,:n_pca]
    Columns = ['pc_'+name + '_' + str(i) for i in range(1,n_pca+1)]
    Cov_Matrix.columns = Columns


    normalized_Cov_Matrix=(Cov_Matrix-Cov_Matrix.min())/(Cov_Matrix.max()-Cov_Matrix.min())
    Transform_Matrix = Cov_Matrix.transpose()
    X = X.drop(['labels'], axis = 1).transpose()
    Final_Data = np.dot(Transform_Matrix, X)
    Final_Data = pd.DataFrame(data = Final_Data.transpose(), columns= Columns)

    return Final_Data

#script
transformation = Transformation()

#Final_Data_S1 = transformation.remove_rows(Final_Data_S1)


Cov_Data_S1 = transformation.remove_features(Final_Data_S2)
Cov_Data_S2 = transformation.remove_features(Final_Data_S1)

#Cov_Data_S1 = transformation.centered_data(Cov_Data_S1)
#Cov_Data_S2 = transformation.centered_data(Cov_Data_S2)

Final_Data_Source = transformation.Transoform_Data(Cov_Data_S1, Cov_Data_S1, 7, 'Source',2)
Final_Data_Target = transformation.Transoform_Data(Cov_Data_S2, Cov_Data_S2, 6, 'Target',2)



#Class Divergence_Calculation
We calculated the divergence between features from source and target datasets using the JSD.

In [8]:
import numpy as np 
import pandas as pd 
from scipy.spatial import distance

class Divergence_Calculation:

  def add_target(self, Final_Data, Cov_Data):
    Final_Data['labels'] = Cov_Data.labels.values
    return Final_Data

  def prob_dist_divergence_initial(self, data, num_pc, name):

    results = data
    column = ['pc_'+ name + '_' + str(i) for i in range(1,num_pc+1)]
    res = pd.DataFrame(data = np.zeros((10,1)), columns=['None'])
    
    for i in column:
      table = pd.DataFrame(data = np.zeros((10,2)), \
                           columns= [i+'0.0', i+'1.0'])
      condit = pd.DataFrame(data = results.groupby('labels')[i])
      for j in range(2):
        x = condit.iloc[j,1]
        table.iloc[:,j] = np.histogram(x, bins = 10)[0] / len(x)   
      res = pd.concat([res, table], axis=1)

    return res

  
  def jsd_final(self, X1_Source_init,X2_Target_init, Final_Data_Target):

    target_prob = Final_Data_Target.labels.value_counts()/len(Final_Data_Target)
    matrix = pd.DataFrame(data = np.zeros((Final_Data_Source.shape[1]-1, Final_Data_Target.shape[1]-1)))
    matrix.columns = [i for i in Final_Data_Target.columns if i!= 'labels']
    matrix.index = [i for i in Final_Data_Source.columns if i != 'labels']
      
    for i in matrix.columns:
      for j in matrix.index:
        matrix.loc[j,i] = sum([target_prob.loc[target_prob.index == k].\
          values[0]*distance.jensenshannon(X1_Source_init[j+str(float(k))], X2_Target_init[i+str(float(k))], 2.0) for k in target_prob.index])
    return matrix

#script
divergence_calculation = Divergence_Calculation()
Final_Data_Source = divergence_calculation.add_target(Final_Data_Source, Cov_Data_S1)
Final_Data_Target = divergence_calculation.add_target(Final_Data_Target, Cov_Data_S2)

X1_Source_init = divergence_calculation.prob_dist_divergence_initial(Final_Data_Source, 7, 'Source')
X2_Target_init = divergence_calculation.prob_dist_divergence_initial(Final_Data_Target, 6, 'Target')

divergence_matrix = divergence_calculation.jsd_final(X1_Source_init,X2_Target_init, Final_Data_Target)



#Class PreMapping
We used Thresholidng to avoid negative transfer and we created the Preferences lists for the mapping procedure.

In [9]:
import numpy as np 
import pandas as pd 

class PreMapping:

  def threshold_select(self, divergence_matrix, threshold):
    divergence_matrix_bool = divergence_matrix > threshold
    indx = []

    for j in range(divergence_matrix.shape[1]):
      Test = True
      for i in range(len(divergence_matrix)):
        if divergence_matrix_bool.iloc[i,j] == False:
          Test = False
      if Test == True:
        indx.append(j)

    new_indx = [i for i in range(divergence_matrix.shape[1]) if i not in indx]
    divergence_matrix = divergence_matrix.iloc[:,new_indx]
    return divergence_matrix

  def preferences(self, divergence_matrix):
    Source_features = [i for i in divergence_matrix.index]
    Target_features = [i for i in divergence_matrix.columns]

    priority_source = {i:list(divergence_matrix.loc[i,:].sort_values().index) for i in Source_features}
    priority_target = {i:list(divergence_matrix.loc[:,i].sort_values().index) for i in Target_features}

    return priority_source, priority_target

#script
preMapping = PreMapping()
#divergence_matrix = preMapping.threshold_select(divergence_matrix, 0.7)
priority_source, priority_target = preMapping.preferences(divergence_matrix)
  

#Class Mapping
We applied the Gale-Shapley Algorithm to map features from both domains based on their divergence values.

In [10]:
import numpy as np 
import pandas as pd 
from collections import defaultdict
class Mapping:

    def __init__(self, men, women):
        '''
        Constructs a Matcher instance.
        Takes a dict of men's spousal preferences, `men`,
        and a dict of women's spousal preferences, `women`.
        '''
        self.M = men
        self.W = women
        self.wives = {}
        self.pairs = []

        # we index spousal preferences at initialization 
        # to avoid expensive lookups when matching
        self.mrank = defaultdict(dict)  # `mrank[m][w]` is m's ranking of w
        self.wrank = defaultdict(dict)  # `wrank[w][m]` is w's ranking of m

        for m, prefs in men.items():
            for i, w in enumerate(prefs):
                self.mrank[m][w] = i

        for w, prefs in women.items():
            for i, m in enumerate(prefs):
                self.wrank[w][m] = i


    def __call__(self):
        return self.match()

    def prefers(self, w, m, h):
        '''Test whether w prefers m over h.'''
        return self.wrank[w][m] < self.wrank[w][h]

    def after(self, m, w):
        '''Return the woman favored by m after w.'''
        i = self.mrank[m][w] + 1    # index of woman following w in list of prefs
        return self.M[m][i]

    def match(self, men=None, next=None, wives=None):
        '''
        Try to match all men with their next preferred spouse.
        
        '''
        if men is None: 
            men = self.M.keys()         # get the complete list of men
        if next is None: 
            # if not defined, map each man to their first preference
            next = dict((m, rank[0]) for m, rank in self.M.items()) 
        if wives is None: 
            wives = {}                  # mapping from women to current spouse
        if not len(men): 
            self.pairs = [(h, w) for w, h in wives.items()]
            self.wives = wives
            return wives
        m, men = list(men)[0], list(men)[1:]
        w = next[m]                     # next woman for m to propose to
        next[m] = self.after(m, w)      # woman after w in m's list of prefs
        if w in wives:
            h = wives[w]                # current husband
            if self.prefers(w, m, h):
                men.append(h)           # husband becomes available again
                wives[w] = m            # w becomes wife of m
            else:
                men.append(m)           # m remains unmarried
        else:
            wives[w] = m                # w becomes wife of m
        return self.match(men, next, wives)

    def map_source(self, Final_Data_Source, Final_Data_Target, Final_Match):

      Final_Data_Source = Final_Data_Source.rename(columns=Final_Match)
      Final_Data_Source = Final_Data_Source[Final_Data_Target.columns]
      return Final_Data_Source

#script
mapping = Mapping(priority_target, priority_source)
Final_Match = mapping.match()
Final_Data_Target = Final_Data_Target[list(Final_Match.values()) + ['labels']]
Final_Data_Source = mapping.map_source(Final_Data_Source, Final_Data_Target, Final_Match)

#Class PostMapping
We added target data in the training to enhance the performance of the model.

In [11]:
import numpy as np 
import pandas as pd 

class PostMapping:

  def shuffle_data(self, Final_Data):
    Final_Data = Final_Data.sample(frac = 1)
    return Final_Data

  def type_change(self, Final_Data):
    Final_Data.labels = Final_Data.labels.astype(float)
    return Final_Data


  def rearrage_data(self, Final_Data_Source, Final_Data_Target, number_rows):
    add_data = Final_Data_Target.iloc[:number_rows,:]
    Final_Data_Target = Final_Data_Target.iloc[number_rows:,:]
    frames = [Final_Data_Source, add_data]
    Final_Data_Source = pd.concat(frames)

    return Final_Data_Source, Final_Data_Target


#script
postMapping = PostMapping()

Final_Data_Source = postMapping.type_change(Final_Data_Source)

Final_Data_Source = postMapping.shuffle_data(Final_Data_Source)
Final_Data_Target = postMapping.shuffle_data(Final_Data_Target)
Data_Target = Final_Data_Target
# Target data added in the training
# 0 days = 0
# 2 days = 178
# 4 days = 355
Final_Data_Source, Final_Data_Target = postMapping.rearrage_data(Final_Data_Source, Final_Data_Target, 355)

#Class Modeling

In [12]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import precision_recall_fscore_support as score
class Modeling:

  def split_train_test(self, Final_Data_Source, Final_Data_Target):
    X_train = Final_Data_Source.drop(['labels'], axis=1)
    y_train = Final_Data_Source['labels']
    X_test = Final_Data_Target.drop(['labels'], axis=1)
    y_test = Final_Data_Target['labels']

    return X_train, y_train, X_test, y_test

  def split_train_val(self, X, y):    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.35, random_state=42)
    return X_train, X_val, y_train, y_val

  def pipelines_def(self):
    pipelines = []
    params = []
    names = []
    #Notice that we tried to balance the data via using the clf__class_weight parameter in the models

    pipelines.append(Pipeline([('clf', DecisionTreeClassifier())])) ## DecisionTreeClassifier
    params.append({'clf__max_features': [None], 'clf__min_samples_split': [2], 'clf__min_samples_leaf':[1],
                  'clf__class_weight': ['balanced']})
    names.append('DecisionTreeClassifier') 



    return pipelines, params, names

  def model(self, pipeline, param, name, X, y):    
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=32)

    grid_obj = GridSearchCV(estimator=pipeline, param_grid=param, cv=cv, scoring='f1_micro', n_jobs=-1)
    grid_obj.fit(X,y)  

    print(name, 'F1-measure:', grid_obj.best_score_)
    estimator = grid_obj.best_estimator_
    estimator.fit(X,y) # train on all training dataset
    return estimator 


  def estimators(self, pipelines, params, names,  X_train, y_train):
    estimators = []
    for idx in range(0,len(pipelines)):    
        estimators.append(self.model(pipelines[idx], params[idx], names[idx], X_train, y_train))
    return estimators


  def evaluate_models(self,estimators, names, X_test, y_test):
   
    for idx, estimator in enumerate(estimators):
      print('\nPerformance of', names[idx])
      y_pred = estimator.predict(X_test)       
      print('\nConfusion matrix\n', confusion_matrix(y_test, y_pred), '\n')    
      print('F1-measure', f1_score(y_test, y_pred, average='micro'), '\n')

      precision, recall, fscore, support = score(y_test, y_pred)

      print('precision: {}'.format(precision))
      print('recall: {}'.format(recall))
      print('fscore: {}'.format(fscore))
      print('support: {}'.format(support))
    return

#script
modeling = Modeling()

X_train, y_train, X_test, y_test = modeling.split_train_test(Final_Data_Source, Final_Data_Target)
X_test_T, y_test_T, X_test_T, y_test_T = modeling.split_train_test(Data_Target, Data_Target)
X_train_S,  X_test_S, y_train_S, y_test_S = modeling.split_train_val(X_test_T, y_test_T                                                                  
                                                                     )
pipelines, params, names = modeling.pipelines_def()
print('Source Training results_without TL')
estimators = modeling.estimators(pipelines, params, names, X_train_S, y_train_S)
print('Source Testing results_without TL')
modeling.evaluate_models(estimators, names, X_test_S, y_test_S)


print('Training results_TL')
estimators = modeling.estimators(pipelines, params, names, X_train, y_train)
print('Testing results_TL')
modeling.evaluate_models(estimators, names, X_test, y_test)

Source Training results_without TL
DecisionTreeClassifier F1-measure: 0.8389075630252101
Source Testing results_without TL

Performance of DecisionTreeClassifier

Confusion matrix
 [[126  10]
 [ 14  37]] 

F1-measure 0.8716577540106952 

precision: [0.9        0.78723404]
recall: [0.92647059 0.7254902 ]
fscore: [0.91304348 0.75510204]
support: [136  51]
Training results_TL
DecisionTreeClassifier F1-measure: 0.9338914606520241
Testing results_TL

Performance of DecisionTreeClassifier

Confusion matrix
 [[125   7]
 [ 11  35]] 

F1-measure 0.898876404494382 

precision: [0.91911765 0.83333333]
recall: [0.9469697  0.76086957]
fscore: [0.93283582 0.79545455]
support: [132  46]


In [13]:
#Thank you